In [73]:
from __future__ import print_function

import argparse
import os
import sys
import numpy as np
from textacy.datasets.supreme_court import SupremeCourt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from time import gmtime, strftime
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data

import xml.etree.ElementTree as ET

In [74]:
model_checkPoint_path = '../ModelCheckpoint/cnn15_leg_and_case_embFixed/'
model_checkPoint_file_name = 'cnn15_leg_and_case_embFixed'
EPOCH_NUM = 100
VALIDATION_SPLIT = 0.1
BATCH_SIZE = 32
MAX_NB_WORDS = 170000
EMBEDDING_DIM = 300
learning_rate = 1e-4

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('which device are we in: ', device) # cuda:0 means we do have a gpu
# create saved path
if not os.path.exists(model_checkPoint_path):
    os.makedirs(model_checkPoint_path)

which device are we in:  cuda:0


In [75]:
# load pre-trained embedding
print('Indexing word vectors ...')
embeddings_index = {}
embedding_path = '../data/GoogleNews-vectors-negative300.txt'
# f = file_io.FileIO('../data/GoogleNews-vectors-negative300.txt', mode='r')
with open(embedding_path, 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
# f.close()
print('Found %s word vectors.' % len(embeddings_index))


Indexing word vectors ...
Found 3000000 word vectors.


### process citation: legislation and case

In [76]:
def get_record_caseName_and_citeID(feature_record):
    # process case name
    case_name_record = feature_record['case_name']
    if case_name_record == None:
        print('We do find a None in case_name')
    if '@' in case_name_record:
        print('We do find a @')
        
    # process cite id
    cite_id_record = feature_record['us_cite_id']
    if cite_id_record == None:
        cite_id_record = 'None'
    if '@' in cite_id_record:
        print('We do find a @')
    
    # prepare for the dictionary key
    output_dict_key = case_name_record+'@'+cite_id_record
    return output_dict_key

# Read from the saved mapping file
def get_caseName_and_citeID_to_savedID_dict():
    saved_mapping_path = '/misc/grice1/yijun/SCOTUS-Embedding/data/'
    saved_mapping_file_name = 'caseName_and_citeID_to_savedID.txt'

    # read the saved mapping file
    with open(saved_mapping_path + saved_mapping_file_name, 'r') as f:
        read_list = f.read().split('\n')[:-1]

    print(type(read_list))
    print('length of read_list: ', len(read_list))

    # build a dict to map case from 8K dateset to the file name
    read_dict = {}
    for line in read_list:
        case_name, cite_id, file_name = line.split('@')
        key = case_name + '@' + cite_id
        read_dict[key] = file_name
        
    return read_dict


In [132]:
# remove single quote
def fix_format_problem_for_specific_file(file_id, text):
    if file_id == '3658': # to fix the & problem at line 2 column 123
        return text.replace('." ', '.  ') 
    elif file_id == '4119': 
        return text.replace('." 1', '.  1')
    elif file_id == '6349': 
        return text.replace('." 1', '.  1').replace('." B', '.  B')
    elif file_id == '7072': 
        return text.replace('.]."', '.]. ')
    elif file_id == '7103': 
        return text.replace('Act."', 'Act. ')
    elif file_id == '7578': 
        return text.replace('York."', 'York. ')
    elif file_id == '7835': 
        return text.replace('");', ' );')
    elif file_id == '8244': 
        return text.replace('little" ', 'little  ')
    elif file_id == '8294': 
        return text.replace('county."', 'county. ')
    elif file_id == '8311': 
        return text.replace('dismissal"', 'dismissal ')
    return text 

def fix_format_problems(file_id, text):
    text = text.replace('& ', '  ') # to fix the & problem at line 10 column 119 in file 1973.legislation10
    text = text.replace('... ."', '... . ') # to fix the & problem at line 16 column 148 in file 3009.legislation10
    return fix_format_problem_for_specific_file(file_id, text)

def replace_sub_string(input_string, start, end, input_class):
    return input_string[:start] + input_class + input_string[end:]

path_UWash_leg_and_case = '/misc/grice1/meyers/appellate_courts/UWash_scotus_output/'
def replace_leg_and_case(file_id, text_before_processed):
    leg_file = path_UWash_leg_and_case + file_id + '.legislation10'
    case_file = path_UWash_leg_and_case + file_id + '.case10'
    start_end_index_leg_and_case_dict = {}
    
    # legislation file
    with open(leg_file, 'r') as leg_file_reader:
        print('processing leg file')
        temp_text = leg_file_reader.read()
        temp_text = fix_format_problems(file_id, temp_text)
        
        tree = ET.fromstring("<root>" + temp_text + "</root>")  # deal with multiple root node
        tree = ET.ElementTree(tree)

        root = tree.getroot()
        for child in root:
            if child.tag == 'citation':
                temp_attrib = child.attrib
#                 print(temp_attrib)
                temp_start = int(temp_attrib['start'])
                temp_end = int(temp_attrib['end'])
#                 print(temp_start)
#                 print(temp_end)
#                 print('segmented text: ', text_before_processed[temp_start:temp_end])
                start_end_index_leg_and_case_dict[(temp_start, temp_end)] = 'legislation'
    
    # case file
    with open(case_file, 'r') as case_file_reader:
        print('processing case file')
        temp_text = case_file_reader.read()
        tree = ET.fromstring("<root>" + temp_text + "</root>")  # deal with multiple root node
        tree = ET.ElementTree(tree)

        root = tree.getroot()
        for child in root:
            if child.tag == 'citation':
                temp_attrib = child.attrib
#                 print(temp_attrib)
                temp_start = int(temp_attrib['start'])
                temp_end = int(temp_attrib['end'])
#                 print(temp_start)
#                 print(temp_end)
#                 print('segmented text: ', text_before_processed[temp_start:temp_end])
                start_end_index_leg_and_case_dict[(temp_start, temp_end)] = 'case'
    
    # sort the dict
    start_end_index_leg_and_case_tuple_list = sorted(start_end_index_leg_and_case_dict.items(), key=lambda pair:pair[0][0], reverse=True)
    # print('start_end_index_leg_and_case_tuple_list: ', start_end_index_leg_and_case_tuple_list)
    
    text_processed = text_before_processed # initialization
    for tuple_pair in start_end_index_leg_and_case_tuple_list:
        tuple_start, tuple_end = tuple_pair[0]
        tuple_class = tuple_pair[1]
        if tuple_class == 'legislation':
            text_processed = replace_sub_string(text_processed, tuple_start, tuple_end, 'legislation')
#             text_processed[tuple_start:tuple_end] = 'legislation'
        elif tuple_class == 'case':
            text_processed = replace_sub_string(text_processed, tuple_start, tuple_end, 'case')
#             text_processed[tuple_start:tuple_end] = 'case'
        else:
            print('This tuple class is not valid, it can only be case or legislation: ', tuple_class)
        
    print('--------------------------------------------------')
    
    return text_processed

In [133]:
print('Processing text dataset')

sc = SupremeCourt()
print(sc.info)

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids

issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
issue_codes.sort()
issue_codes = [str(ic) for ic in issue_codes]

labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))

# get mapping dict between UWash dataset and local file name (id)
caseName_and_citeID_to_savedID_dict = get_caseName_and_citeID_to_savedID_dict()

for record in sc.records():
    text_record = record[0]
    feature_record = record[1]
    
    if feature_record['issue'] == None: # some cases have None as an issue
        labels.append(labels_index['-1'])
    else:
        labels.append(labels_index[feature_record['issue'][:-4]])
    
    # get file id to process leg and case
    dict_key = get_record_caseName_and_citeID(feature_record)
    file_id = caseName_and_citeID_to_savedID_dict[dict_key]
    # print('processing: ', file_id)
    
    # process citation: legislation and case
    text_processed = replace_leg_and_case(file_id, text_record)
    
    # print('text_processed: ', text_processed)
    texts.append(text_processed)
    
#     break

print('Found %s texts.' % len(texts))
print('Found %s labels.' % len(labels_index))

Processing text dataset
{'name': 'supreme_court', 'site_url': 'http://caselaw.findlaw.com/court/us-supreme-court', 'description': 'Collection of ~8.4k decisions issued by the U.S. Supreme Court between November 1946 and June 2016.'}
<class 'list'>
length of read_list:  8419
processing:  0000
processing leg file
processing case file
--------------------------------------------------
processing:  0001
processing leg file
processing case file
--------------------------------------------------
processing:  0002
processing leg file
processing case file
--------------------------------------------------
processing:  0003
processing leg file
processing case file
--------------------------------------------------
processing:  0004
processing leg file
processing case file
--------------------------------------------------
processing:  0005
processing leg file
processing case file
--------------------------------------------------
processing:  0006
processing leg file
processing case file
------

processing leg file
processing case file
--------------------------------------------------
processing:  0116
processing leg file
processing case file
--------------------------------------------------
processing:  0117
processing leg file
processing case file
--------------------------------------------------
processing:  0118
processing leg file
processing case file
--------------------------------------------------
processing:  0119
processing leg file
processing case file
--------------------------------------------------
processing:  0120
processing leg file
processing case file
--------------------------------------------------
processing:  0121
processing leg file
processing case file
--------------------------------------------------
processing:  0122
processing leg file
processing case file
--------------------------------------------------
processing:  0123
processing leg file
processing case file
--------------------------------------------------
processing:  0124
processing

processing:  0196
processing leg file
processing case file
--------------------------------------------------
processing:  0197
processing leg file
processing case file
--------------------------------------------------
processing:  0198
processing leg file
processing case file
--------------------------------------------------
processing:  0199
processing leg file
processing case file
--------------------------------------------------
processing:  0200
processing leg file
processing case file
--------------------------------------------------
processing:  0201
processing leg file
processing case file
--------------------------------------------------
processing:  0202
processing leg file
processing case file
--------------------------------------------------
processing:  0203
processing leg file
processing case file
--------------------------------------------------
processing:  0204
processing leg file
processing case file
--------------------------------------------------
processing

--------------------------------------------------
processing:  0317
processing leg file
processing case file
--------------------------------------------------
processing:  0318
processing leg file
processing case file
--------------------------------------------------
processing:  0319
processing leg file
processing case file
--------------------------------------------------
processing:  0320
processing leg file
processing case file
--------------------------------------------------
processing:  0321
processing leg file
processing case file
--------------------------------------------------
processing:  0322
processing leg file
processing case file
--------------------------------------------------
processing:  0323
processing leg file
processing case file
--------------------------------------------------
processing:  0324
processing leg file
processing case file
--------------------------------------------------
processing:  0325
processing leg file
processing case file
----------

processing:  0450
processing leg file
processing case file
--------------------------------------------------
processing:  0451
processing leg file
processing case file
--------------------------------------------------
processing:  0452
processing leg file
processing case file
--------------------------------------------------
processing:  0453
processing leg file
processing case file
--------------------------------------------------
processing:  0454
processing leg file
processing case file
--------------------------------------------------
processing:  0455
processing leg file
processing case file
--------------------------------------------------
processing:  0456
processing leg file
processing case file
--------------------------------------------------
processing:  0457
processing leg file
processing case file
--------------------------------------------------
processing:  0458
processing leg file
processing case file
--------------------------------------------------
processing

...

--------------------------------------------------
processing:  8034
processing leg file
processing case file
--------------------------------------------------
processing:  8035
processing leg file
processing case file
--------------------------------------------------
processing:  8036
processing leg file
processing case file
--------------------------------------------------
processing:  8037
processing leg file
processing case file
--------------------------------------------------
processing:  8038
processing leg file
processing case file
--------------------------------------------------
processing:  8039
processing leg file
processing case file
--------------------------------------------------
processing:  8040
processing leg file
processing case file
--------------------------------------------------
processing:  8041
processing leg file
processing case file
--------------------------------------------------
processing:  8042
processing leg file
processing case file
----------

processing case file
--------------------------------------------------
processing:  8111
processing leg file
processing case file
--------------------------------------------------
processing:  8112
processing leg file
processing case file
--------------------------------------------------
processing:  8113
processing leg file
processing case file
--------------------------------------------------
processing:  8114
processing leg file
processing case file
--------------------------------------------------
processing:  8115
processing leg file
processing case file
--------------------------------------------------
processing:  8116
processing leg file
processing case file
--------------------------------------------------
processing:  8117
processing leg file
processing case file
--------------------------------------------------
processing:  8118
processing leg file
processing case file
--------------------------------------------------
processing:  8119
processing leg file
processing

processing case file
--------------------------------------------------
processing:  8195
processing leg file
processing case file
--------------------------------------------------
processing:  8196
processing leg file
processing case file
--------------------------------------------------
processing:  8197
processing leg file
processing case file
--------------------------------------------------
processing:  8198
processing leg file
processing case file
--------------------------------------------------
processing:  8199
processing leg file
processing case file
--------------------------------------------------
processing:  8200
processing leg file
processing case file
--------------------------------------------------
processing:  8201
processing leg file
processing case file
--------------------------------------------------
processing:  8202
processing leg file
processing case file
--------------------------------------------------
processing:  8203
processing leg file
processing

processing case file
--------------------------------------------------
processing:  8272
processing leg file
processing case file
--------------------------------------------------
processing:  8273
processing leg file
processing case file
--------------------------------------------------
processing:  8274
processing leg file
processing case file
--------------------------------------------------
processing:  8275
processing leg file
processing case file
--------------------------------------------------
processing:  8276
processing leg file
processing case file
--------------------------------------------------
processing:  8277
processing leg file
processing case file
--------------------------------------------------
processing:  8278
processing leg file
processing case file
--------------------------------------------------
processing:  8279
processing leg file
processing case file
--------------------------------------------------
processing:  8280
processing leg file
processing

--------------------------------------------------
processing:  8359
processing leg file
processing case file
--------------------------------------------------
processing:  8360
processing leg file
processing case file
--------------------------------------------------
processing:  8361
processing leg file
processing case file
--------------------------------------------------
processing:  8362
processing leg file
processing case file
--------------------------------------------------
processing:  8363
processing leg file
processing case file
--------------------------------------------------
processing:  8364
processing leg file
processing case file
--------------------------------------------------
processing:  8365
processing leg file
processing case file
--------------------------------------------------
processing:  8366
processing leg file
processing case file
--------------------------------------------------
processing:  8367
processing leg file
processing case file
----------

### prepare train/dev/test set

In [ ]:
# vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

# with tf.device('/gpu:0'):
padded_data = pad_sequences(sequences)

MAX_SEQUENCE_LENGTH = padded_data.shape[1]

labels = to_categorical(np.asarray(labels))
    
print('Shape of padded_data ndarray:', padded_data.shape)
print('Shape of label ndarray:', labels.shape)

In [ ]:
# prepare embedding matrix -> tensor
num_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
# split the data into a training set and a validation set
x_train_ndarray, x_test_ndarray, y_train_ndarray, y_test_ndarray = train_test_split(padded_data, labels, test_size=VALIDATION_SPLIT, random_state=42)
x_train_ndarray, x_val_ndarray, y_train_ndarray, y_val_ndarray = train_test_split(x_train_ndarray, y_train_ndarray, test_size=VALIDATION_SPLIT, random_state=42)

# to tensor and to gpu
x_train = torch.from_numpy(x_train_ndarray).to(device, dtype=torch.long)
# x_train = x_train.unsqueeze(1)
y_train = torch.from_numpy(y_train_ndarray).to(device, dtype=torch.long)

x_val = torch.from_numpy(x_val_ndarray).to(device, dtype=torch.long)
# x_val = x_val.unsqueeze(1)
y_val = torch.from_numpy(y_val_ndarray).to(device, dtype=torch.long)

x_test = torch.from_numpy(x_test_ndarray).to(device, dtype=torch.long)
# x_test = x_test.unsqueeze(1)
y_test = torch.from_numpy(y_test_ndarray).to(device, dtype=torch.long)

In [ ]:
# Dataloader
dataset_train = data.TensorDataset(x_train,y_train)
dataloader_train = data.DataLoader(dataset_train, batch_size=4, shuffle=True)
dataset_val = data.TensorDataset(x_val,y_val)
dataloader_val = data.DataLoader(dataset_val, batch_size=4, shuffle=False)
dataset_test = data.TensorDataset(x_test,y_test)
dataloader_test = data.DataLoader(dataset_test, batch_size=4, shuffle=False)

In [ ]:
dataiter_temp = iter(dataloader_train)
images_temp, labels_temp = dataiter_temp.next()
# images_temp = images_temp.unsqueeze(1)
print(images_temp.size())
print(labels_temp.size())

### Model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.embedding = nn.Embedding(num_words, EMBEDDING_DIM)
        self.embedding.weight = nn.Parameter(torch.from_numpy(embedding_matrix).float())
        self.embedding.weight.requires_grad = False
        
        self.conv_module = nn.Sequential(
            
            nn.Conv1d(300,128,5),
            nn.ReLU(),
            nn.MaxPool1d(5),
            nn.Dropout(0.25),
            
            nn.Conv1d(128,128,5),
            nn.ReLU(),
            nn.MaxPool1d(5),
            nn.Dropout(0.25),
            
            nn.Conv1d(128,128,5),
            nn.ReLU(),
#             nn.MaxPool1d(128),
            nn.Dropout(0.5),
                
        )
        
        self.dense_module = nn.Sequential(
            nn.Linear(128, 128),
#             nn.Dropout(0.5),
            nn.Linear(128, len(labels_index)),
#             nn.Softmax()
        )

#         self.conv1 = nn.Conv2d(3, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 5 * 5, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(0, 2, 1)
        
        x = self.conv_module(x)
#         print('after conv: ', x.size())
        x, _  = torch.max(x, 2) # global max pooling
#         print('after max: ', x.size())
        x = self.dense_module(x)
        
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 16 * 5 * 5)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
        return x

In [ ]:
model = Net().to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
model

In [ ]:
def get_accuracy(model, dataloader):
    correct = 0
    total = 0
    model.eval()
    for data in dataloader:
        images, labels = data
        outputs = model(images)
        _, outputs = torch.max(outputs, 1) # get the class index
        _, labels = torch.max(labels, 1)

        total += labels.size(0)
        correct += (outputs == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

def save_model(model, epoch, best):
    if best == False:
        torch.save(model.state_dict(), 
                   model_checkPoint_path+model_checkPoint_file_name+'_'+epoch+'.pth')
    else:
        torch.save(model.state_dict(), 
                   model_checkPoint_path+model_checkPoint_file_name+'_best.pth')
    print('Model saved. Epoch: %d, Best: %r' % (epoch, best))
    return epoch

In [ ]:
best_accuracy_val = 0
epochs_of_best_models_list = []

for epoch in range(EPOCH_NUM):  # loop over the dataset multiple times
    
    # train
    running_loss = 0.0
    for i, data_train in enumerate(dataloader_train, 0):
        model.train()
        # get the inputs; data is a list of [inputs, labels]
        inputs_train, labels_train = data_train
#         print(inputs_train.size())
#         print(labels_train.size())
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs_train = model(inputs_train)
#         print(outputs_train.size())
#         print(labels_train.size())
#         print(torch.max(labels_train, 1)[1])
#         print('outputs_train: ', outputs_train)
        
        loss = criterion(outputs_train, torch.max(labels_train, 1)[1])
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if (i+1) % 400 == 0:    # print every 400 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    
    # validation
    if epoch % 1 == 0: # may change if needed
        print('-------------- Evaluating --------------')
        accuracy_val = get_accuracy(model, dataloader_val)
        print('Evaluation Accuracy: %f %%' % (accuracy_val))      
        if accuracy_val > best_accuracy_val:
            best_accuracy_val = accuracy_val
            epoch_of_best_model = save_model(model, epoch, best=True)
            epochs_of_best_models_list.append(epoch_of_best_model)
        if epoch % 5 == 0: # save the model every 5 epochs.
            save_model(model, epoch, best=False)
        
print('Finished Training')
print('epochs_of_best_models_list (latest last): ', epochs_of_best_models_list)


In [ ]:
print('-------------- testing --------------')
# load best model
best_model = Net().to(device)
best_model.load_state_dict(torch.load(model_checkPoint_path+model_checkPoint_file_name+'_best.pth', 
                                      map_location=device))
accuracy_test = get_accuracy(best_model, dataloader_test)
print('Testing Accuracy: %f %%' % (accuracy_test))